### Solving Optimization Problems

In [16]:
def solve_master():

    # update and solve the model
    master.update()
    master.write('master.lp')
    master.optimize()

    z_hat = {}

    for i in potential_hubs:
        for q in capacity_levels:
            z_hat[(i, q)] = z[i, q].X

    master_opt_val = master.ObjVal

    return z_hat, master_opt_val

In [17]:
def solve_dsp(z_hat):

    # suppress Gurobi output
    environment = gp.Env(empty = True)
    environment.setParam("OutputFlag", 0)
    environment.start()

    # model
    dsp = gp.Model('dsp', env = environment)

    # variables + objective function
    alpha = dsp.addVars(commodities, name = "alpha")
    u = dsp.addVars(potential_hubs, commodities, lb = 0, name = "u")
    v = dsp.addVars(potential_hubs, lb = 0, name = "v")

    obj = gp.quicksum(alpha[k] for k in commodities) - gp.quicksum(z_hat[i, q] * u[i, k] for q in capacity_levels for i in potential_hubs for k in commodities) - gp.quicksum(new_hub_capacities[i, q] * z_hat[i, q] * v[i] for q in capacity_levels for i in potential_hubs)

    dsp.setObjective(obj)

    # model sense
    dsp.ModelSense = GRB.MAXIMIZE

    # constraints
    for i in potential_hubs:
        for j in potential_hubs:
            for k in commodities:

                if i != j:
                    dsp.addConstr(alpha[k] - u[i, k] - u[j, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))
                dsp.addConstr(alpha[k] - u[i, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, i, k))

    # update and solve the model
    dsp.update()
    dsp.write('dsp.lp')
    dsp.optimize()

    alpha_vals = {}
    u_vals = {}
    v_vals = {}

    for i in potential_hubs:
        for k in commodities:
            v_vals[i] = v[i].X
            alpha_vals[k] = alpha[k].X
            u_vals[i, k] = u[i, k].X

    dsp_opt_val = dsp.ObjVal

    return alpha_vals, u_vals, v_vals, dsp_opt_val

In [18]:
def benders_decomp():
    
    upper_bound = np.inf
    t = 0
    terminate = False

    # print(f"upper_bound at iteration {t}: {upper_bound}")

    while (terminate == False) & (t <= 1000):
        z_hat, master_opt_val = solve_master()

        if master_opt_val == upper_bound:
            # print(f"Done at {t} steps")
            terminate = True
        else:
            alpha_new, u_new, v_new, dsp_opt_val = solve_dsp(z_hat)
            master.addConstr(eta >= gp.quicksum(alpha_new[k] for k in commodities) 
                                  - gp.quicksum(z[i, q] * u_new[i, k] for q in capacity_levels for i in potential_hubs for k in commodities)
                                  - gp.quicksum(new_hub_capacities[i, q] * z[i, q] * v_new[i] for q in capacity_levels for i in potential_hubs))

            if dsp_opt_val + setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat) < upper_bound:
                upper_bound = dsp_opt_val + setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat)

        t += 1
        # print(f"upper_bound at iteration {t}: {upper_bound}")

    return z_hat, upper_bound

### Benders decomposition

In [19]:
def setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat):

    sum = 0

    for i in potential_hubs:
        for q in capacity_levels:
            sum += fixed_set_up_costs_with_capacity_level[i, q] * z_hat[i, q]

    return sum

In [20]:
def benders_decomp():
    
    upper_bound = np.inf
    t = 0
    terminate = False

    # print(f"upper_bound at iteration {t}: {upper_bound}")

    while (terminate == False) & (t <= 1000):
        z_hat, master_opt_val = solve_master()

        if master_opt_val == upper_bound:
            # print(f"Done at {t} steps")
            terminate = True
        else:
            alpha_new, u_new, v_new, dsp_opt_val = solve_dsp(z_hat)
            master.addConstr(eta >= gp.quicksum(alpha_new[k] for k in commodities) 
                                  - gp.quicksum(z[i, q] * u_new[i, k] for q in capacity_levels for i in potential_hubs for k in commodities)
                                  - gp.quicksum(new_hub_capacities[i, q] * z[i, q] * v_new[i] for q in capacity_levels for i in potential_hubs))

            if dsp_opt_val + setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat) < upper_bound:
                upper_bound = dsp_opt_val + setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat)

        t += 1
        # print(f"upper_bound at iteration {t}: {upper_bound}")

    return z_hat, upper_bound